# ЗАНЯТИЕ 2.5. АНСАМБЛИ МОДЕЛЕЙ

Загружаем библиотеки

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#устраним ошибки со шрифтами
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['DejaVu Sans']

# Часть 1. Бэггинг

## Описание задачи

Используем данные страхового подразделения BNP Paribas из соревнования

https://www.kaggle.com/c/bnp-paribas-cardif-claims-management

Решается задача классификации страховых случаев:
    1. Случаи, требующие дополнительных документов для подтвердения (0)
    2. Случаи, которые можно подтверждать автоматически на основе имеющейся информации (1)

## Загрузка данных

In [2]:
data = pd.read_csv('train.csv')

Уменьшим размер данных для ускорения обучения, возмем случайную подвыборку 20% данных со стратификацией

In [3]:
# from sklearn.model_selection import StratifiedShuffleSplit

# random_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=777)

# for train_index, test_index in random_splitter.split(data, data.target):
#     data = data.iloc[test_index]

Разбиение на обучение и hold-out тест 70/30. Данных достаточно много, поэтому можно принебречь честной кросс-валидацией и оценивать модель на тесте

In [4]:
from sklearn.model_selection import ShuffleSplit
splitter = ShuffleSplit(n_splits=1, test_size=0.3, random_state=777)

for train_index, test_index in splitter.split(data, data.SalePrice):
    d_train = data.iloc[train_index]
    d_test = data.iloc[test_index]
    
    y_train = data['SalePrice'].iloc[train_index]
    y_test = data['SalePrice'].iloc[test_index]

In [5]:
#d_train.SalePrice

## Первичный анализ

Размер датасета

In [6]:
data.shape

(1460, 81)

Распределение значений таргета (event rate)

In [7]:
#data.SalePrice.value_counts()/ len(data)

## Предобработка данных

Находим категориальные признаки

Чтобы в разы не увеличивать число признаков при построении dummy, будем использовать категориальные признаки с < 30 уникальных значений

In [8]:
cat_feat = list(data.dtypes[data.dtypes == object].index)

#закодируем пропущенные значений строкой, факт пропущенного значения тоже может нести в себе информацию
data[cat_feat] = data[cat_feat].fillna('nan')

#отфильтруем непрерывные признаки
num_feat = [f for f in data if f not in (cat_feat + ['ID', 'SalePrice'])]

cat_nunique = d_train[cat_feat].nunique()
print(cat_nunique)
cat_feat = list(cat_nunique[cat_nunique < 30].index)

MSZoning          5
Street            2
Alley             2
LotShape          4
LandContour       4
Utilities         1
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          7
Exterior1st      15
Exterior2nd      16
MasVnrType        4
ExterQual         4
ExterCond         5
Foundation        6
BsmtQual          4
BsmtCond          4
BsmtExposure      4
BsmtFinType1      6
BsmtFinType2      6
Heating           6
HeatingQC         4
CentralAir        2
Electrical        5
KitchenQual       4
Functional        6
FireplaceQu       5
GarageType        6
GarageFinish      3
GarageQual        5
GarageCond        5
PavedDrive        3
PoolQC            3
Fence             4
MiscFeature       4
SaleType          9
SaleCondition     6
dtype: int64


In [9]:
cat_feat

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [10]:
from sklearn.metrics import auc, roc_curve
from sklearn.linear_model import LogisticRegression

## Композиции моделей одного семейства

### Будем использовать решающие деревья

1. Неустойчивы к входным данным
2. Склонны к переобучению
3. Быстро обучаются

=> отличный выбор для построения композиций

**Создаем признаки для "деревянных" моделей**

1. Заменяем пропуски на специальное значение -999, чтобы деревья могли их отличить
3. Создаем дамми-переменные для категорий

In [14]:
dummy_train = pd.get_dummies(d_train[cat_feat], columns=cat_feat)
dummy_test = pd.get_dummies(d_test[cat_feat], columns=cat_feat)

dummy_cols = list(set(dummy_train) & set(dummy_test))

dummy_train = dummy_train[dummy_cols]
dummy_test = dummy_test[dummy_cols]


X_train = pd.concat([d_train[num_feat].fillna(-999),
                     dummy_train], axis=1)

X_test = pd.concat([d_test[num_feat].fillna(-999),
                     dummy_test], axis=1)

Обучаем решающее дерево

Немного ограничим глубину и минимальное кол-во объектов в листе для уменьшения переобучения

In [15]:
from sklearn.tree import DecisionTreeRegressor

clf_tree = DecisionTreeRegressor(max_depth=15, min_samples_leaf=20)
clf_tree.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=20,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [16]:
clf_tree.score(X_train, y_train)

0.8225400455887872

In [17]:
clf_tree.score(X_test, y_test)

0.8091866981086849

#### Считаем ROC AUC

In [19]:
def calc_auc(y, y_pred, plot_label='', prin=True):
    fpr, tpr, _ = roc_curve(y, y_pred)
    auc_val = auc(fpr, tpr)
    if prin:
        print('ROC AUC: {0:.4f}'.format(auc_val))
    if plot_label:
        plt.plot(fpr, tpr, label=plot_label)
        plt.xlabel('FPR')
        plt.ylabel('TPR')
    return auc_val

Предсказываем вероятность класса 1 и считаем ROC AUC

In [20]:
y_pred_test = clf_tree.predict(X_test)
y_pred_train = clf_tree.predict(X_train)

print('Train:')
calc_auc(y_train, y_pred_train, 'train')
print('Test:')
calc_auc(y_test, y_pred_test, 'test')
plt.legend();

Train:


ValueError: multiclass format is not supported

### Бэггинг

Используем готовый алгоритм из sklearn

In [52]:
from sklearn.ensemble import BaggingRegressor

bag_clf = BaggingRegressor(n_estimators=20, base_estimator=clf_tree, n_jobs=-1)
bag_clf.fit(X_train, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=15, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=20,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=20, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [53]:
bag_clf.score(X_train, y_train)

0.8303573099499755

In [54]:
bag_clf.score(X_test, y_test)

0.8681040652366913

In [55]:
# y_pred_test = bag_clf.predict(X_test)
# y_pred_train = bag_clf.predict(X_train)

# print('Train:')
# calc_auc(y_train, y_pred_train, 'train')
# print('Test:')
# calc_auc(y_test, y_pred_test, 'test')
# plt.legend();

# Часть 2. Случайный лес

Бэггинг + случайные подпространства = случайный лес

**Важные гиперпараметры алгоритма**

a. Параметры деревьев
    1. criterion - критерий построения дерева
    2. max_depth - максимальная глубина дерева (обычно 10-20, больше глубина -> больше риск переобучения)
    3. min_samples_leaf - минимальное число объектов в листе (обычно 20+, больше объектов -> меньше риск переобучения)

b. Параметры леса
    1. n_estimators - кол-во деревьев (чем больше тем лучше)
    2. max_features - число признаков случайного подпространства
    3. bootstrap - использовать ли бэггинг
    4. n_jobs - кол-во потоков для одновременного построения деревьев (большая прибавка к скорости на многоядерных процах)

In [56]:
'минимальное число объектов в листе'.upper()

'МИНИМАЛЬНОЕ ЧИСЛО ОБЪЕКТОВ В ЛИСТЕ'

In [59]:
from sklearn.ensemble import RandomForestRegressor
clf_rf = RandomForestRegressor(n_estimators=100, max_depth=15, min_samples_leaf=20, max_features=0.8, n_jobs=-1)

clf_rf.fit(X_train, y_train)    

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=15,
           max_features=0.8, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=20, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [60]:
clf_rf.score(X_train, y_train)

0.837671585028116

In [61]:
clf_rf.score(X_test, y_test)

0.8790429361504788

In [62]:
# y_pred_rf_test = clf_rf.predict_proba(X_test)[:, 1]
# y_pred_rf_train = clf_rf.predict_proba(X_train)[:, 1]

# print('Train:')
# calc_auc(y_train, y_pred_train, 'train')
# print('Test:')
# calc_auc(y_test, y_pred_test, 'test')
# plt.legend();

In [63]:
# y_pred_rf_test = clf_rf.predict(X_test)[:, 1]


#### Важность признаков

В sklearn - усредненное по всем деревьям в ансамбле кол-во сплитов по признаку, взвешенное на прирост информации (Information gain) и долю объектов в вершине, в которой производится этот сплит

Это не единственный вариант, см здесь:

https://medium.com/@ceshine/feature-importance-measures-for-tree-models-part-i-47f187c1a2c3

Важности признаков случайного леса лежат в артибуте **feature\_importances\_**

In [64]:
imp = pd.Series(clf_rf.feature_importances_, index=X_train.columns)
imp.sort_values(ascending=False)


OverallQual             0.620135
GrLivArea               0.092020
GarageCars              0.085706
TotalBsmtSF             0.032237
BsmtFinSF1              0.027197
GarageArea              0.021879
1stFlrSF                0.017609
LotArea                 0.013232
YearBuilt               0.012803
ExterQual_TA            0.010686
BsmtQual_Ex             0.007776
KitchenQual_Ex          0.006628
Fireplaces              0.006182
TotRmsAbvGrd            0.005286
FullBath                0.004825
YearRemodAdd            0.004108
2ndFlrSF                0.003702
MasVnrArea              0.003142
GarageType_Attchd       0.002900
GarageYrBlt             0.002084
ExterQual_Ex            0.002010
CentralAir_Y            0.001540
LotShape_Reg            0.001514
WoodDeckSF              0.001368
ExterQual_Gd            0.001236
CentralAir_N            0.001186
OpenPorchSF             0.001015
KitchenQual_Gd          0.000869
MSSubClass              0.000802
KitchenQual_TA          0.000798
          

# Часть 3. Композиции моделей разных типов

### Линейная комбинация моделей разного типа

Смешаем дерево и логистическую регрессию

**Создаем признаки для лог регрессии**

1. Заменяем пропуски на медианы
2. Создаем индикаторы пропущенных значений
3. Создаем дамми-переменные для категорий
4. Нормируем признаки

In [65]:
from sklearn.preprocessing import StandardScaler

train_median = d_train[num_feat].median()

X_train_lin = pd.concat([d_train[num_feat].fillna(train_median),
                     d_train[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_train], axis=1)

X_test_lin = pd.concat([d_test[num_feat].fillna(train_median),
                     d_test[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_test], axis=1)

scaler = StandardScaler()
scaler.fit(X_train_lin[num_feat])

X_train_lin[num_feat] = scaler.transform(X_train_lin[num_feat])
X_test_lin[num_feat] = scaler.transform(X_test_lin[num_feat])

Обучим логистическую регрессию

In [73]:
clf_lr = LogisticRegression(penalty='l1', C=0.1)

clf_lr.fit(X_train_lin, y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [77]:
clf_lr.score(X_train_lin, y_train)

0.05675146771037182

In [76]:
# y_pred_lin_test = clf_lr.predict_proba(X_test_lin)[:, 1]
# y_pred_lin_train = clf_lr.predict_proba(X_train_lin)[:, 1]

# print('Train:')
# calc_auc(y_train, y_pred_lin_train, 'train')
# print('Test:')
# calc_auc(y_test, y_pred_lin_test, 'test')
# plt.legend();

Будем строить линейную комбинацию вида

$y = \alpha y_1 + (1 - \alpha) y_2$

Параметр $\alpha$ переберем по сетке от 0 до 1, оценивая качество на тестовой выборке

In [75]:
# aucs = []
# alpha_space = np.linspace(0, 1, 100)
# for alpha in alpha_space:
#     y_pred_weight = alpha * y_pred_lin_test + (1 - alpha) * y_pred_rf_test
#     aucs.append(calc_auc(y_test, y_pred_weight, prin=False))
# aucs = np.array(aucs)    

# max_ind = np.where(aucs == aucs.max())[0]
# alpha = alpha_space[max_ind]

# plt.plot(alpha_space, aucs)
# plt.plot(alpha_space[max_ind], aucs[max_ind], 'o', c='r')
# plt.xlabel('alpha')
# plt.ylabel('auc')

# #итоговое взвешенное предсказание
# y_pred_weight = alpha * y_pred_lin_test + (1 - alpha) * y_pred_rf_test

NameError: name 'y_pred_rf_test' is not defined

Сравним 3 метода (приблизим график ROC кривой, чтобы увидеть разницу)

In [ ]:
# print('Weighted:')
# calc_auc(y_test, y_pred_weight, 'weighted')
# print('Log regression:')
# calc_auc(y_test, y_pred_lin_test, 'LR')
# print('Random forest:')
# calc_auc(y_test, y_pred_rf_test, 'RF')
# plt.legend();
# plt.xlim(0.2, 0.5)
# plt.ylim(0.5, 0.8)

### Стэкинг

#### Средние значения таргета

Создадим новые признаки, на основе категориальных переменных. Каждому уникальному значению $V$ переменной $X_i$ сопоставим среднее значение таргета среди всех объектов, у которых переменная $X_i$ принимает значение $V$ 

Новый признак со средними значением таргета в категории можно считать за предсказание вер-ти класса 1 простого классификатора "усреднения"

Опишем класс этого классификатора

In [80]:
class MeanClassifier():
    def __init__(self, col):
        self._col = col
        
    def fit(self, X, y):
        self._y_mean = y.mean()
        self._means = y.groupby(X[self._col].astype(str)).mean()

    def predict_proba(self, X):
        new_feature = X[self._col].astype(str)\
            .map(self._means.to_dict())\
            .fillna(self._y_mean)
        return np.stack([1-new_feature, new_feature], axis=1)

Делаем предсказания по фолдам кросс-валидации. **Главное не допустить утечки информации!**

Опишем функцию для стекинга

In [81]:
def get_meta_features(clf, X_train, y_train, X_test, stack_cv):
    meta_train = np.zeros_like(y_train, dtype=float)
    meta_test = np.zeros_like(y_test, dtype=float)
    
    for i, (train_ind, test_ind) in enumerate(stack_cv.split(X_train, y_train)):
        
        clf.fit(X_train.iloc[train_ind], y_train.iloc[train_ind])
        meta_train[test_ind] = clf.predict_proba(X_train.iloc[test_ind])[:, 1]
        meta_test += clf.predict_proba(X_test)[:, 1]
    
    return meta_train, meta_test / stack_cv.n_splits

### Стекинг нескольких моделей
0. Средние значения
1. Random Forest
2. Log reg
3. SVM

Посмотрим, какое качество дает линейный SVM

для совместимости с общим кодом стекинга немного модифицируем класс SVM

In [83]:
from sklearn.svm import LinearSVC

def norm(x):
    return (x - x.min()) / (x.max() - x.min())

class SVMWrapper(LinearSVC):
    def predict_proba(self, X):
        df = norm(self.decision_function(X))
        return np.stack([1-df, df], axis=1)

clf_svm = SVMWrapper(C=0.001)    
clf_svm.fit(X_train_lin, y_train)

y_pred_svm_test = clf_svm.predict_proba(X_test_lin)[:, 1]
y_pred_svm_train = clf_svm.predict_proba(X_train_lin)[:, 1]

# print('Train:')
# calc_auc(y_train, y_pred_lin_train, 'train')
# print('Test:')
# calc_auc(y_test, y_pred_lin_test, 'test')
# plt.legend();

Теперь получим мета признаки для 3х моделей:
* SVM
* Logreg
* Random Forest
и средних значений по каждой категориальной переменной

In [85]:
from sklearn.model_selection import StratifiedKFold

stack_cv = StratifiedKFold(n_splits=10, random_state=555)

meta_train = []
meta_test = []
col_names = []

print('mean features...')
for c in cat_nunique.index.tolist():
    clf = MeanClassifier(c)
    
    meta_tr, meta_te = get_meta_features(clf, d_train, y_train, d_test, stack_cv)

    meta_train.append(meta_tr)
    meta_test.append(meta_te)
    col_names.append('mean_pred_{}'.format(c))

print('SVM features...')
meta_tr, meta_te = get_meta_features(clf_svm, X_train_lin, y_train, X_test_lin, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('svm_pred')

print('LR features...')
meta_tr, meta_te = get_meta_features(clf_lr, X_train_lin, y_train, X_test_lin, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('lr_pred')

print('RF features...')
meta_tr, meta_te = get_meta_features(clf_rf, X_train, y_train, X_test, stack_cv)

meta_train.append(meta_tr)
meta_test.append(meta_te)
col_names.append('rf_pred')

mean features...


/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


SVM features...


ValueError: shape mismatch: value array of shape (546,196) could not be broadcast to indexing result of shape (546,)

In [86]:
X_meta_train = pd.DataFrame(np.stack(meta_train, axis=1), columns=col_names)
X_meta_test = pd.DataFrame(np.stack(meta_test, axis=1), columns=col_names)

#### Стэкинг мета-признаков с помощью LR

Используем регуляризованную лог регрессию в качестве алгоритма второго уровня

In [87]:
clf_lr_meta = LogisticRegression(penalty='l2', C=1)

clf_lr_meta.fit(X_meta_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [88]:
y_pred_meta_test = clf_lr_meta.predict_proba(X_meta_test)[:, 1]

calc_auc(y_test, y_pred_meta_test, 'test')
plt.legend();

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


ValueError: multiclass format is not supported

#### Посмотрим на коэффициенты объединяющей линейной модели

Получим интерпретацию общей модели

In [89]:
pd.Series(clf_lr_meta.coef_.flatten(), index=X_meta_train.columns).plot(kind='barh')

ValueError: Wrong number of items passed 23392, placement implies 43

# Домашняя работа

#### Простая
1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
3. Построить случайный лес, вывести важность признаков
4. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
5. Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
6. Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)
7. В качестве решения:
    Jupyter notebook с кодом, комментариями и графиками

#### Средняя
0. Все то же, что и в части 1, плюс:
1. Попробовать другие оценки важности переменных, например Boruta
http://danielhomola.com/2015/05/08/borutapy-an-all-relevant-feature-selection-method/#comments
3. Изучить extremely randomized trees (ExtraTreesRegressor в sklearn), сравнить с Random Forest
4. Проводить настройку гиперпараметров для моделей первого уровня в стекинге (перебирать руками и смотреть на CV или по сетке: GridSearchCV, RandomizedSearchCV)
5. Попробовать другие алгоритмы второго уровня
6. Сделать сабмиты на kaggle (минимум 3: отдельные модели vs стекинг), сравнить качество на локальной валидации и на leaderboard
7. В качестве решения:
    * Jupyter notebook с кодом, комментариями и графиками
    * сабмит на kaggle (ник на leaderboard)